In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import functions

# count dfs for the TL case, input: mean annual, daily 

In [9]:
%%time


# langtang
folder_path = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_daily/4landcover_mean/mustang_climate_cut'
elevation = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_mustang.csv')[['cellnr2','band_data']] 
landcover = 'landcover4'
location = 'mustang'

column = 'dfs'
result_df = pd.DataFrame()


# Iterate over the files in the folders
for folder_name in os.listdir(folder_path):
    # skip all csvs and . files
    if folder_name.endswith('.csv'):
        continue
    if folder_name.startswith('.'):
        continue 
    
    file_path = os.path.join(folder_path, folder_name, 'Sediment.out') #create a path to file 
    
    if os.path.isfile(file_path): #check if its there 
        # read file 
        output_df = pd.read_csv(file_path)
        # count dfs per given time 
        output_df = functions.count_dfs_per_time(output_df, column, 'year')
        
        # take the COUNT column 
        name_column = output_df['count']
        # rename the columns 
        column_name = f'{column}_{folder_name}'
        result_df[column_name] = name_column
        result_df.columns = [col[-3:] for col in result_df.columns]


annaul_df_count = functions.add_elevation_to_df_count(result_df, elevation)


output = '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_daily/output_mean_annual/'
annaul_df_count.to_csv(output + f'{location}_annual_df_count_annualmean_{landcover}.csv')

<timed exec>:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


default land cover. monthly data: same
CPU times: user 46.1 s, sys: 1.96 s, total: 48 s
Wall time: 48.7 s


In [ ]:
result_df